In [29]:
import dataAck
import portfolio
import time
import hashlib
import sys
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [30]:
##GET PREDICTIONS OF ALL MODELS IN A PORTFOLIO

allPortfolios = portfolio.getPortfolios()
allPortfolios

[{'benchmark': 'VO',
  'description': 'VO 8-13-17',
  'key': '7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f'}]

In [31]:
allModels = []
modelsInPortfolio = {}
for portfolioInfo in allPortfolios:
    print(portfolioInfo)
    models = portfolio.getModelsByKey(portfolio.getPortfolioModels(portfolioInfo["key"]))
    modelsInPortfolio[portfolioInfo["key"]] = models
    allModels += models
    for model in models:
        print(model.describe())
    

{'key': '7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f', 'description': 'VO 8-13-17', 'benchmark': 'VO'}
((10, ('XLV', 31, None, 14), 2, 'VO'), 10, 50)
((10, ('SHY', 30, None, None), 3, 'VO'), 10, 25)
((10, ('SHY', 30, None, None), 2, 'VO'), 10, 25)
((10, ('TIP', 24, None, 15), 2, 'VO'), 10, 25)
((22, ('IWF', 4, None, None), 3, 'VO'), 22, 25)


In [32]:
tickersRequired = []
tickersTraded = []
for mod in allModels:
    print(mod.describe())
    if mod.inputSeries.targetTicker not in tickersRequired:
        tickersRequired.append(mod.inputSeries.targetTicker)
    if mod.inputSeries.series.ticker not in tickersRequired:
        tickersRequired.append(mod.inputSeries.series.ticker)
    if mod.inputSeries.targetTicker not in tickersTraded:
        tickersTraded.append(mod.inputSeries.targetTicker)
    
pulledData, validTickers = dataAck.downloadTickerData(tickersRequired)

joinedData = dataAck.joinDatasets([pulledData[ticker] for ticker in pulledData])

((10, ('XLV', 31, None, 14), 2, 'VO'), 10, 50)
((10, ('SHY', 30, None, None), 3, 'VO'), 10, 25)
((10, ('SHY', 30, None, None), 2, 'VO'), 10, 25)
((10, ('TIP', 24, None, 15), 2, 'VO'), 10, 25)
((22, ('IWF', 4, None, None), 3, 'VO'), 22, 25)
TIP
VO
SHY
IWF
XLV


In [33]:
from google.cloud import datastore, storage, logging
import time
import params
import hashlib
import pandas as pd
def downloadAggregatePredictions(model):
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            query = datastore_client.query(kind=params.aggregatePrediction)
            
            query.add_filter('modelHash', '=', hashlib.sha224((str(model.describe())).encode('utf-8')).hexdigest())
            retrievedPredictions = list(query.fetch())
            days = []
            predictions = []
            for pred in retrievedPredictions:
                days.append(pred["predictionDay"])
                predictions.append(pred["aggregatePrediction"])
            
            return pd.DataFrame(predictions, index=days, columns=[str(model.describe())]).sort_index()
        except:
            time.sleep(10)
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))

In [35]:
aggregateReturns = None
aggregatePredictions = None
for model in allModels:
    preds = downloadAggregatePredictions(model).tz_localize(None)
    dailyFactorReturn = dataAck.getDailyFactorReturn(model.inputSeries.targetTicker, joinedData)
    transformedPreds = preds.join(dailyFactorReturn).dropna()
    returnStream = pd.DataFrame(transformedPreds.apply(lambda x:x[0] * x[1], axis=1), columns=[portfolio.getModelHash(model)])
    preds.columns = [portfolio.getModelHash(model)]
    if aggregateReturns is None:
        aggregateReturns = returnStream
        aggregatePredictions = preds
    else:
        aggregateReturns = aggregateReturns.join(returnStream)
        aggregatePredictions = aggregatePredictions.join(preds)

In [36]:
aggregateReturns

,78afec28d6c4e55102b09d8b6b179a3508e2b03fb46d2e8858c14fdc,0b84734d97e005d3a2a3a06d82cb793ec1f7557573e3bc032fb7fdf3,2c69141675447782e2a74036a7cbd8fb5fc08476cebf557bd6e787a6,341fe3e8eb88c9fbdd779c6c8727762f99fc1043383dd8242da044e9,11912f289e86b30f1af1b7272e773732dd099e3069cdf79f1c2ba47e
2006-11-06,-0.002369,0.004737,0.000000,0.000000,-0.002369
2006-11-07,-0.003772,0.003772,0.003772,0.003772,-0.001886
2006-11-08,0.002349,-0.002349,-0.002349,-0.002349,-0.000000
2006-11-09,-0.002197,0.001099,0.002197,0.002197,0.000000
2006-11-10,-0.001879,0.000000,0.000000,0.001879,-0.000940
2006-11-13,0.000000,-0.005314,0.000000,0.010628,-0.005314
2006-11-14,0.002011,-0.004021,0.000000,0.000000,-0.004021
2006-11-15,0.000000,-0.000000,-0.000000,-0.000000,-0.000000
2006-11-16,0.000000,-0.000924,-0.000924,0.000000,-0.000462
2006-11-17,-0.000616,-0.001231,0.000000,0.000000,0.000000


In [37]:
aggregatePredictions

,78afec28d6c4e55102b09d8b6b179a3508e2b03fb46d2e8858c14fdc,0b84734d97e005d3a2a3a06d82cb793ec1f7557573e3bc032fb7fdf3,2c69141675447782e2a74036a7cbd8fb5fc08476cebf557bd6e787a6,341fe3e8eb88c9fbdd779c6c8727762f99fc1043383dd8242da044e9,11912f289e86b30f1af1b7272e773732dd099e3069cdf79f1c2ba47e
2006-11-06,-0.5,1.0,0.0,0.0,-0.500000
2006-11-07,-1.0,1.0,1.0,1.0,-0.500000
2006-11-08,-1.0,1.0,1.0,1.0,0.000000
2006-11-09,-1.0,0.5,1.0,1.0,0.000000
2006-11-10,-1.0,0.0,0.0,1.0,-0.500000
2006-11-13,0.0,-0.5,0.0,1.0,-0.500000
2006-11-14,0.5,-1.0,0.0,0.0,-1.000000
2006-11-15,0.5,-1.0,-1.0,-1.0,-1.000000
2006-11-16,0.0,-1.0,-1.0,0.0,-0.500000
2006-11-17,-0.5,-1.0,0.0,0.0,0.000000


In [53]:
def storePortfolioAllocation(portfolioKey, predictionDay, algorithmWeights, tickerAllocation, transformedAlgoPrediction, shouldReturn = False):
    toUpload = {}
    toUpload["portfolio"] = portfolioKey
    toUpload["predictionDay"] = predictionDay
    
    for item in algorithmWeights:
        toUpload["algo_weight_" + item] = algorithmWeights[item]
    
    for item in transformedAlgoPrediction:
        toUpload["algo_" + item] = transformedAlgoPrediction[item]
    
    for item in tickerAllocation:
        toUpload["ticker_" + item] = tickerAllocation[item]
        
    
    
    ##UPLOAD ORGANISM OBJECT
    while True:
        try:
            datastoreClient = datastore.Client('money-maker-1236')
            #HASH DIGEST
            predictionHash = hashlib.sha224((str(portfolioKey) + " " + str(toUpload["predictionDay"])).encode('utf-8')).hexdigest()
            key = datastoreClient.key(params.portfolioAllocation, predictionHash) #NEED TO HASH TO ENSURE NON-OVERLAPPING PREDICTIONS
            organismToStore = datastore.Entity(key=key)
            organismToStore.update(toUpload)
            if shouldReturn == False:
                datastoreClient.put(organismToStore)
            else:
                return organismToStore
            break
        except:
            print("UPLOAD ERROR:", str(sys.exc_info()))
            time.sleep(10)

In [54]:
import hrpPortfolioOpt as hrp
def produceHRPPredictions(aggregateReturns, windowSize, startIndex, maxWindowSize = False):
    hrpReturns = pd.DataFrame([])
    historicalWeights = pd.DataFrame([])
    i = windowSize
    if startIndex is not None:
        i = len(aggregateReturns) - windowSize - startIndex
    while i < len(aggregateReturns):
        corr = None
        cov = None
        if maxWindowSize == False:
            corr = (aggregateReturns[:i]).corr()
            cov = (aggregateReturns[:i]).cov()
        else:
            corr = (aggregateReturns[i-windowSize:i]).corr()
            cov = (aggregateReturns[i-windowSize:i]).cov()
        weights = hrp.getHRP(cov, corr)
    #     display(weights)
    #     display(aggregateReturns[i+windowSize:i+windowSize+1])
        todayReturn = aggregateReturns[i:i+1] * weights
    #     display(todayReturn)
        sumReturn = pd.DataFrame(todayReturn.apply(lambda x:sum(x), axis=1))
        hrpReturns = pd.concat([hrpReturns, sumReturn])
        thisWeights = pd.DataFrame([[weights[item] for item in weights.index]], index=sumReturn.index, columns=weights.index.tolist())
        historicalWeights = pd.concat([historicalWeights, thisWeights])
        i += 1
    return hrpReturns, historicalWeights

In [57]:
import importlib
importlib.reload(portfolio)

<module 'portfolio' from '/home/prohb125/walkforwardTrader/portfolio.py'>

In [55]:
def storeHistoricalAllocations(portfolioKey, modelsInPortfolio, historicalWeights, aggregatePredictions):

    aggregatePredictions = aggregatePredictions.dropna()
    allocationsToStore = []
    ##ITERATE THROUGH DAYS TO CALCULATE NET POSITION
    for i in range(len(historicalWeights)):
        netPosition = {}
        weights = historicalWeights.iloc[i]
        transformedAlgoPrediction = {}
        for model in modelsInPortfolio:
            if model.inputSeries.targetTicker not in netPosition:
                netPosition[model.inputSeries.targetTicker] = 0.0
            netPosition[model.inputSeries.targetTicker] += weights[portfolio.getModelHash(model)] * aggregatePredictions.loc[historicalWeights.index[i]][portfolio.getModelHash(model)]
            transformedAlgoPrediction[portfolio.getModelHash(model)] = weights[portfolio.getModelHash(model)] * aggregatePredictions.loc[historicalWeights.index[i]][portfolio.getModelHash(model)]
        allocationsToStore.append(storePortfolioAllocation(portfolioKey, historicalWeights.index[i], weights.to_dict(), netPosition, transformedAlgoPrediction, shouldReturn=True))
    portfolio.storeManyItems(allocationsToStore)

In [56]:
##GENERATE WEIGHTS FOR PORTFOLIO
storeHistorical = True
allocationsToStore = []
for portfolioKey in modelsInPortfolio:
    hrpReturns, historicalWeights = produceHRPPredictions(aggregateReturns[[portfolio.getModelHash(model) for model in modelsInPortfolio[portfolioKey]]], 22, startIndex=None, maxWindowSize=True)
    if storeHistorical == True:
        storeHistoricalAllocations(portfolioKey, modelsInPortfolio[portfolioKey], historicalWeights, aggregatePredictions)
    print(portfolioKey, historicalWeights.iloc[-1])
    todayWeight = historicalWeights.iloc[-1]
    netPosition = {}
    transformedAlgoPrediction = {}
    for model in modelsInPortfolio[portfolioKey]:
        if model.inputSeries.targetTicker not in netPosition:
            netPosition[model.inputSeries.targetTicker] = 0.0
        netPosition[model.inputSeries.targetTicker] += todayWeight[portfolio.getModelHash(model)] * portfolio.getAggregatePredictionForModelDaily(model, joinedData)
        transformedAlgoPrediction[portfolio.getModelHash(model)] = todayWeight[portfolio.getModelHash(model)] * portfolio.getAggregatePredictionForModelDaily(model, joinedData)
    print(portfolioKey, netPosition)
    allocationsToStore.append(storePortfolioAllocation(portfolioKey, portfolio.getToday(), todayWeight.to_dict(), netPosition, transformedAlgoPrediction, shouldReturn=True))
portfolio.storeManyItems(allocationsToStore)

KeyboardInterrupt: 

In [ ]:
portfolio.getPortfolioAllocations("7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f", portfolio.getToday())